In [1]:
# include("../src/BoundaryConditions.jl")
# include("../src/Configurations.jl")
# include("../src/Ensembles.jl")
# using .Ensembles

using ParallelTemperingMonteCarlo
using StaticArrays

In [2]:
x = NVT(10)

NVT(10)

In [3]:
function testtype(testinput::NVT)
    println("true")
end

function testmove(xinput::X) where X <: MoveType
    println("true")
end

testmove (generic function with 1 method)

In [4]:
testtype(x)

true


In [5]:
xmove = atommove()

typeof(xmove)

atommove

In [6]:
testmove(xmove)

true


In [8]:
bc = SphericalBC(radius=2.0)
v1 = SVector(1., 2., 3.)
conf = Config{3}([v1,v1,v1],bc)

envars_nvt = set_ensemble_variables(conf,NVT(1))
xx = typeof(envars_nvt)

NVTVariables

In [9]:
typeof(envars_nvt.index)

Int64

In [10]:
length(envars_nvt.trial_move) 

3

In [16]:
envars_nvt.trial_move = atom_displacement(SVector(0.,0.,0.),0.5,bc)

3-element SVector{3, Float64} with indices SOneTo(3):
  0.06797443479555543
 -0.15170255100268498
 -0.24777240290674468

In [17]:
envars_nvt.trial_move = atom_displacement(xmove,0.5,bc)

3-element SVector{3, Float64} with indices SOneTo(3):
 -0.3169025133439916
  0.5828375346844659
 -0.2222299593885947